In [183]:
import json
import httpx
from parsel import Selector
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [253]:
def review_and_ratings(url):
    with httpx.Client(http2=True, follow_redirects=True) as client:
        resp = client.get(url)
    sel = Selector(text=resp.text)
    
    selected_reviews = sel.xpath(r'//span[@class=" raw__09f24__T4Ezm"]').getall()
    selected_ratings = sel.xpath(r'//div[@class="yelp-emotion-9tnml4"]').getall() # Changed from css-14g69b3 -- 05/02/2024
    
    selected_reviews = selected_reviews[4:] # Get 11 reviews per page
    new_len = len(selected_reviews)
    selected_ratings = selected_ratings[2:new_len+2]
    return selected_reviews, selected_ratings

In [269]:
def get_all_biz(url, stop):
    reviews = []
    ratings = []
    for i in range(0,stop*10,10):
        rev,rat = review_and_ratings(url+str(i))
        reviews += rev
        ratings += rat
    return reviews, ratings

In [256]:
# Test for GB eats
rev,rat = get_all_biz("https://www.yelp.com/biz/gb-eats-great-barrington?start=", 2)

100%|██████████| 5/5 [00:16<00:00,  3.21s/it]


In [260]:
# Goal is 7000, aprx 2.5k
starter_urls = [("https://www.yelp.com/biz/gb-eats-great-barrington?start=", 17),
                ("https://www.yelp.com/biz/baba-louies-sourdough-pizza-great-barrington-3?start=", 35),
                ("https://www.yelp.com/biz/soco-creamery-great-barrington?start=", 19),
                ("https://www.yelp.com/biz/rio-cafe-great-barrington?start=", 7),
                ("https://www.yelp.com/biz/prairie-whale-great-barrington?osq=Restaurants&start=", 35),
                ("https://www.yelp.com/biz/the-well-great-barrington?osq=Restaurants&start=", 11),
                ("https://www.yelp.com/biz/aegean-breeze-restaurant-great-barrington-2?osq=Restaurants&start=", 20),
                ("https://www.yelp.com/biz/the-bistro-box-great-barrington?osq=Restaurants&start=", 18),
                ("https://www.yelp.com/biz/marketplace-kitchen-table-great-barrington?osq=Restaurants&start=", 11),
                ("https://www.yelp.com/biz/barrington-brewery-and-restaurant-great-barrington?osq=Restaurants&start=", 39),
                ("https://www.yelp.com/biz/xicohtencatl-great-barrington?osq=Restaurants&start=", 29),
                ("https://www.yelp.com/biz/pleasant-and-main-housatonic?osq=Restaurants&start=", 21)]

In [270]:
def get_all(urls=starter_urls):
    reviews = []
    ratings = []
    for item in tqdm(urls):
        url,stop = item
        try:
            rev,rat = get_all_biz(url, stop)
        except:
            print("Failed at: "+url)
            return reviews, ratings
        if len(rev) != len(rat):
            print("Error!!!")
            exit()
        reviews += rev
        ratings += rat
    return reviews, ratings

In [272]:
rev,rat = get_all()

100%|██████████| 12/12 [12:46<00:00, 63.92s/it]


In [308]:
def extract_text_from_html(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    text = soup.text.strip()
    return text

def extract_rating_from_html(html_string):
    rating_regex = r'aria-label="(\d+(?:\.\d+)?)\sstar\srating"'
    text = re.findall(rating_regex, html_string)
    return text

In [321]:
reviews = []
ratings = []
sentiment = []
for i in range(len(rev)):
    star = extract_rating_from_html(rat[i])
    if star is not None:
        reviews.append(extract_text_from_html(rev[i]))
        ratings.append(float(star[0]))
        if float(star[0]) > 3.0:
            sentiment.append(1)
        else:
            sentiment.append(0)
        
#reviews = [extract_text_from_html(x) for x in rev]
#ratings = [extract_rating_from_html(x) for x in rat]

In [322]:
save_df = pd.DataFrame(data={'review': reviews, 'rating': ratings, 'sentiment': sentiment}) # 1 for positive, 0 for negative

In [323]:
save_df.to_csv("./Data/2.5k_reviews.csv", index=False)